## IMDB Web Scrapper

William Egan - wve204 | Tinatin Nikvashvili - tn709 | Nathan Griffin - nlg297

### Abstract

We have constructed a web scraper that pulls instances from IMDB of movies, and stores them in a dataframe. This algorithm can be modified to pull moves based on genre, date released, lanaguage used, and many other metrics. The program gathers a wide range of features for each film, including financial and personnel data as well as information about the movie itself. This collection of features allows data scientists to pursue a wide range of data mining and analytical work, as we demonstrate in this notebook.

### Motivation

IMDB offers downloadable datasets, but all of these are limited in scope (link to said datasets here: https://www.imdb.com/interfaces/). None of these contain any more than 8 features, and it would be complicated data munging task in and off itself to merge all of these into a single data frame. There are also a limited number of features offered by these, including financial information about the film, which is a feature of great interest for most data-driven business problems relating to film.

These limited channels of access inspired us to employ our python skills to develop a web scraper for pulling exactly the information we want from webpages for films. Along with determining all the features we want, we can also choose what subset of films we would like to pull from based off of IMDB's own advanced search tool. A program that can effectively pull data sets of movies with any desired features or movie type would be useful for anyone who wants to do any kind of data analysis on films. We believe the particular features we have chosen to be especially interesting to perform analysis and data mining methods on. 

Another facet of our inspiration for this project is our dissatisfaction with the system of critic reviews and ratings. These metrics are biased and inconsistent, both across the spectrum of different critics and even within individual critics’ histories of publication. We will propose a new, data-driven method based on the data sets we can scrape from IMDB, and discuss the kinds of predictions we hope to be able to perform with this information. [MAYBE MOVE THIS TO THE END]

### Step 1: Extract urls from IMBD search
[IMDB Search](https://www.imdb.com/search/title/)


The first step in this application is to take an input search (link above) and return a list of urls: one for each movie that satisfies those parameters. To implement this function, we will need the help of a few libraries: BeautifulSoup, urllib, and webbrowser (for debugging). We first entered a search into IMDB's advanced search tool. In particular, we entered a search for all English-language feature films released in 2018. We notice that the total number of movies that match the search is listed at the top, and that there are 50 movies per page. Therefore, we know two important variables right off the bat: how many urls we expect to return and how many webpages we will have to iterate over to get them all. 

The function, therefore, will begin by creating a BeautifulSoup object (i.e. a parsed HTML file) of the search url by opening the search url with urllib. Right away, we extract the total number of films returned from this query using the find_all method in BeautifulSoup, replacing the comma with nothing, and converting the resulting string to an integer. We then print the number for the users reference. From here, we need to understand how to move through each page of the query. By looking at page two, we see that it is, fortunately, pretty simple. By just appending "&start=n" where n is any number (1 in our case), to the search url, we can look at a page with 50 entries, starting at the nth entry. Getting a list of these numbers is a simple list comprehension. 

From there, we iterate through each of these urls, creating a new BeautifulSoup object each time, and grabbing all links from the page using the get and findall methods. From the list of links, we look for a specific string 'title/tt' that corresponds to webpages for movie profiles, add it to each if the ursl in the final url list, checking to make sure it is not already there. Once it iterates through each page and adds all urls, a test is run to ensure that the number of movies in the search matches the length of the final url list. 

There was an issue initially with the length of the list exceeding the number of movies in the search, so some debugging was required.  In some cases, there was only one extra link, while in others there was more than 100. The webbrowser library was used to open every 50 links to see which page was adding extra information. In hindsight, a more efficient option may have been to print out how many items were added to the list in each loop and look for values over 50. Regardless, the issue was that some movies had a link to sequels/prequels included with them. This did not show up on the webpage, but was included in the html file. Furthermore, each valid url was listed 3 seperate times, one with an extra directory in the url. Since these were the only title links with a length of 4 when split on '/', that is ultimately how we filter out for valid urls. This method also avoids checking for duplicate values in the list twice for a better optimized runtime.

In [1]:
#Import relevant libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import requests
from time import sleep
from random import randint
import pandas as pd
import numpy as np
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
def url_extractor(search_url):
    #setting up initial BeautifulSoup object from websearch 
    init_resp = urllib.request.urlopen(search_url)
    init_soup = BeautifulSoup(init_resp, 'html.parser')
    #extract the number of films that the query returned. Used to confirm at end and generate each url 
    number_of_films = int(str(init_soup.find_all('div', class_='desc')[0].find_all('span')[0]).split(' ')[2].replace(',',''))
    print(number_of_films)
    #each page has 50 movies, so setting up a list to iterate through the pages, set up blank list to store final urls
    iterative_urls = [i for i in range(1,number_of_films, 50)]
    url_list = []
    #loop through the 50-spaced interger values to generate entire list of needed search urls 
    for i in iterative_urls:
        # set url
        url = search_url + '&start=' + str(i)
        #set up the BeautifulSoup object for this specific page of the search
        resp = urllib.request.urlopen(url)
        soup = BeautifulSoup(resp, 'html.parser')
        #generating list of all links on this page
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        # printing out where we are in the query to monitor efficiency
        print('Running query from {} to {}'.format(i, i+49))
        # checking each link in each search page for title/tt keyword
        for link in links:
            if 'title/tt' in link:
                #when the length is 4 of the split title url, that means it is part of query 
                #when the length is 3, it means that the movie is ancillary to the actual search (prequel/sequel)
                if len(link.split('/')) == 4:
                # format the resulting url in the correct manner and appending it to final list 
                    title_link = 'https://www.imdb.com' + '/' + link.split('/')[1] + '/' + link.split('/')[2] + '/?ref_=adv_li_i'
                    if not title_link in url_list:
                        url_list.append(title_link)
                    else: 
                        continue 
                else: 
                    continue
            else:
                continue
    # Final test to make sure that the length of query equals the length of returned list and returning the final list
    if len(url_list) == number_of_films:
        print('All urls have been extracted successfully')
    else: 
        print('WARNING: The number of films in this query was {}, but {} urls were returned'.format(number_of_films, len(url_list)))
    return(url_list)

### Step 2: Web Scraper

In this part of the project we use web scraping to gather data about all of the query result movies from IMDB's web page in a format that will be usable for analysis. In particular, we want to collect information about movie title, genres, country, language, filming location, production company, runtime, budget information, release date, directors, writers, stars acting in the movie, rating, review count and finally mpaa ratings.

Since our goal is to scrape information from multiple webpages with the same html structure, first we had to understand the structure of a movie profile page on IMDB's website. To do this, we used Chrome's Developer tools to inspect the structure of only one page. By right clicking and hitting inspect you can see the HTML line that corresponds to the part of webpage you clicked on. There are many HTML lines nested within each tag, and for each feature we found unique identifiers of information we wanted to extract. 

Then, in order to extract the contents of the webpage, we download the pages we wanted to scrape by using the request library. The library makes a get request to IMDB's server and downloads the HTML contents of a given web page. After we wun the request using requests.get method, we get back a response object that has a status_code, which equals 200 if the page is downloaded successfully. This enables us to include error catching into our code and return a warning whenever web pages are not downloaded successfully (the case when status_code does not equal 200).  

BeautifulSoup has a functionality that allows for parsing through captured HTML data based on the HTML tags. HTML also has class properties, that give HTML elements names and makes it easier to identify relevant text during scraping. One element can have multiple classes. We can extract the relevant information from tags by specifying class using find_all method to find all instances of a tag in a webpage. For example in line 39 we search for any div tag that has a class of 'see-more inline canwrap'. Find_all returns a list, therefore we used list indexing to extract relevant text by using text method.  Additionally, as all tags are nested, we had to move through levels to get to specific text by first selecting all elements at the top level. For example to gather information about genres we first extract text from any div tag that has class being 'see-more inline canwrap' and then extract text for h4 tag with class_ = 'inline'. Once we are able to uniquely identify how to get each piece of information:

* we can loop through multiple urls
* place get requests within the loop for each page
* convert the response's html content to beautiful soup object
* extract all containers from this object by using find_all method, if there is no information than continue to the next variable
* use list indexing to access information
* and save this information into pandas’ data frame.

At the end of this process, we have a data frame with each row representing information on each movie scraped and each column corresponding to the variable we scraped. 

One of the issues we had to deal with was not having all of the features present on each web page. For example, not all movies have information about genres, language, writers and etc. Therefore, we used try and except statements to scrape information when present and continue running the code when the information was missing.

In [3]:
def scraper(urls):
    '''Scrape information about movies from imdbs website

    Input
    =====
    urls: list of urls
    
    Output
    ======
    movies: dataframe containning as rows movies we scraped and columns the features scraped
    
    '''
    
    #Create a dataframe to store scraped data in (features as columns)
    features = ['genre_0','genre_1','genre_2', 'genre_3','country','language','filming_locs','production_co','runtime',
    'budget','gross_usa','release','director_0','director_1','rating','star_0','star_1','star_2','star_3','writer_0',
    'writer_1','writer_2','review_count','title','open_week','cumulative', 'mpaa_rating']
    
    #number of rows of data frame = # of movies scraped, # of columns = # of features
    movies = pd.DataFrame(data = np.empty((len(urls), len(features))),
                         columns= features)
    
    #data set is initialized as nan
    movies[:] = np.nan
    
    #create a list of rating we want to collect
    rating_list = ['   G', '   PG', 'PG-13', '   R', 'NC-17', 'Not Rated', 'Unrated', 
                   'TV-Y', 'TV-Y7', 'TV-G', 'TV-PG','TV-14', 'TV-MA']   
    
    count = 1
    #loop through urls and scrape relevant features
    for idx,url in enumerate(urls):
        count += 1
        
        resp = requests.get(url)#request content of the webpage and store in resp

        #Pause the loop but takes a long time
        #sleep(randint(8,15))

        #Error if  status codes is not 200
        if resp.status_code != 200:
            warn('Request: {}; Status code: {}'.format(request, response.status_code))

        soup = BeautifulSoup(resp.text, 'html.parser')#use pythons built in html parser
        
        #collect mpaa ratings
        mpaa = soup.find_all('div', class_ = 'title_wrapper')[0].find_all('div', class_ = 'subtext')[0].text
        for i in rating_list:
            if i in mpaa:
                movies.loc[idx, 'mpaa_rating'] = i
                
        #get all genres
        genres_spec = soup.find_all('div', class_ = 'see-more inline canwrap')
        for i in range(len(genres_spec)):
            if 'Genres' in genres_spec[i].find('h4', class_ = 'inline').text:
                genres = soup.find_all('div', class_ = 'see-more inline canwrap')[i].find_all('a')
                movies.loc[idx,'genre_0'] = genres[0].text
                try:
                    movies.loc[idx,'genre_1']= genres[1].text
                except:
                    pass
                try:
                    movies.loc[idx,'genre_2'] = genres[2].text
                except:
                    pass
                try:
                    movies.loc[idx,'genre_3'] = genres[3].text
                except:
                    pass

        #Get country, language, filming location, production co, budget info variables
        other_specs = soup.find_all('div', class_ = 'txt-block')
        for i in range(len(other_specs)):
            if other_specs[i].find('h4', class_ = 'inline') is None:
                pass
            elif 'Country' in other_specs[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'country'] = other_specs[i].a.text
            elif 'Language' in other_specs[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'language'] = other_specs[i].a.text
            elif 'Filming Locations' in other_specs[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'filming_locs'] = other_specs[i].a.text
            elif 'Production Co' in other_specs[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'production_co'] =other_specs[i].a.text
            elif 'Runtime' in other_specs[i].find('h4', class_ = 'inline').text:
                movies.loc[idx, 'runtime'] = other_specs[i].time.text
            elif 'Budget' in other_specs[i].find('h4', class_ = 'inline').text:
                try:
                    movies.loc[idx,'budget']= other_specs[i].text
                except: 
                    pass
            elif 'Gross USA' in other_specs[i].find('h4', class_ = 'inline').text:
                try:
                    movies.loc[idx,'gross_usa'] = other_specs[i].text
                except:
                    pass
            elif 'Opening Weekend USA' in other_specs[i].find('h4', class_ = 'inline').text:
                try:
                    movies.loc[idx,'open_week'] = other_specs[i].text
                except:
                    pass
            elif 'Cumulative Worldwide Gross' in other_specs[i].find('h4', class_ = 'inline').text:
                try:
                    movies.loc[idx,'cumulative'] = other_specs[i].text
                except:
                    pass

        #get release date
        try:
            movies.loc[idx,'release'] = soup.find_all('div', class_ = 'subtext')[0].find_all('a', title = "See more release dates")[0].text
        except:
            pass

        #Get director, writer and stars    
        movie_containers = soup.find_all('div', class_ = 'credit_summary_item')
        for i in range(len(movie_containers)):
            if 'Director' in movie_containers[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'director_0'] = movie_containers[i].find_all('a')[0].text
                try:
                    movies.loc[idx,'director_1']= movie_containers[i].find_all('a')[1].text
                except:
                    pass
            if 'Writer' in movie_containers[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'writer_0']=movie_containers[i].find_all('a')[0].text
                try:
                    movies.loc[idx,'writer_1']=movie_containers[i].find_all('a')[1].text
                except:
                    pass
                try:
                    movies.loc[idx,'writer_2']=movie_containers[i].find_all('a')[2].text
                except:
                    pass

            if 'Star' in movie_containers[i].find('h4', class_ = 'inline').text:
                movies.loc[idx,'star_0']=movie_containers[i].find_all('a')[0].text
                try:
                    movies.loc[idx,'star_1']=movie_containers[i].find_all('a')[1].text
                except:
                    pass
                try:
                    movies.loc[idx,'star_2']=movie_containers[i].find_all('a')[2].text
                except:
                    pass
                try:
                    movies.loc[idx,'star_3']=movie_containers[i].find_all('a')[3].text
                except:
                    pass

        #Get raitng, review count and the title of the movie
        try:
            movies.loc[idx,'rating']=soup.find_all('span',{'itemprop':'ratingValue'})[0].text
        except:
            pass
        try:
            movies.loc[idx,'review_count']=soup.find_all('span',{'itemprop':'ratingCount'})[0].text
        except:
            pass
        movies.loc[idx,'title']=soup.find_all('title')[0].text
        
        
        if count%100 == 0:
            print('Extracted {}'.format(count)) 
        
        
    return movies

### Step 3: Cleaning the Data

The final step of the project is to clean the data set and prepare it for data analysis. When the data is read in from the scraper, beautiful soup captures other elements, of the html code such as line breaks and tab characters that restricts our ability to interpret our data, which we will refer to as 'bad characters'. Our goal in the cleaning of the data is three fold: All columns in the data frame should be of the appropriate data type with the aforementioned 'bad characters', all place holders/category extenders should be removed, and categorical features encoded when necessary. To do so we break the cleaning process into three functions: 

* `clean_data` 
* `encode_genre`
* `adjust_years`

clean_data has three primary functionalities:
* The first is to remove all bad characters from columns. Numerical information to be cleaned includes the films' runtime, budget, gross_usa earnings, rating out of 10, number of reviews, opening week sales and cumulative earnings. All the numerical columns are cleaned and cast as floats. Title and release date are also cleaned to remove bad characters. 
* The second is to clean place holder variables and Nans. Place holder categorical variables are replaced with an empty string and Nans are filled with zeros. 
* For the the purpose of this analysis, we drop rows that do not have financial data (open_week, cumulative, etc.) since these are key features in our analysis later

We chose to encode the categorical variable of genre. This is done by using the encode_genre function. Based on IMDBs website description, there is a reasonable number of genres to encode, so this is a viable task. We use the get_dummies function in pandas in conjunction with aggregating of genre to create a binary column for each genere label.

The last step in the process is the adjust_years funtion. For the purpose of our analysis we only want the years the movie was released. It is important to note that this procedure can be extended to involve months if there is analysis that requires this.  

In [4]:
def clean_data(df):
    num_cols = ['runtime', 'budget', 'gross_usa', 'rating', 'review_count', 'open_week', 'cumulative']
    output = df.copy()
    
    # removing all strings from numeric columns 
    for col in num_cols:
        output[col] = (output[col].str.replace('\D+', '')).astype("float64")
        
    # cleaning null/place holder values   
    output = output.drop(['filming_locs'], axis = 1)
    output = output.fillna(0)
    output.replace(to_replace = "See full cast & crew", value = "", inplace = True)
    output.replace(to_replace = "[0-9][0-9]* more credits*", value = "", inplace = True, regex=True)
    
    # removing 'bad' string from title and date (still need to fix datetime)
    output['title'] = output['title'].str[0:-14]
    output['release']
    
    result = output.loc[(output[['budget', 'gross_usa', 'open_week', 'cumulative']] != 0).any(axis = 1)]

    return result

In [5]:
def encode_genre(df):
    # encoding genre as dummie variables
    output = pd.get_dummies(df, prefix_sep='_', columns = ['genre_0', 'genre_1', 'genre_2', 'genre_3'], drop_first=True)

    #retreives all columns with 'genre'
    genre_ls_all = output.filter(like='genre_').columns.tolist()

    # aggregates genre buckets into single bucket
    genre_ls = []
    for item in genre_ls_all:
        if item[9:] not in genre_ls:
            genre_ls.append(item[9:])

    for col in genre_ls:
        output[col] = output.filter(like = col).sum(axis = 1)
        
    for col in genre_ls:
        output[col] = output[col].clip(upper = 1)

    output = output[output.columns.drop(list(output.filter(regex='genre')))]
    
    return output

In [23]:
def adjust_years(df):
    df = df.reset_index(drop = True)
    year_df = df['release'].str.extract(r'(\b\d{4}\b)')
    year_df.columns = ['Year']
    result = pd.concat([year_df, df.drop(['release'], axis = 1)], axis = 1)
    result = result.fillna(0)
    result['Year'] = result.Year.astype('int')
    
    return result
    

### Execution

In [28]:
#url = url_extractor('https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-03-01')
#url = url_extractor('https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&languages=en')
url = url_extractor('https://www.imdb.com/search/title/?groups=top_250')

250
Running query from 1 to 50
Running query from 51 to 100
Running query from 101 to 150
Running query from 151 to 200
Running query from 201 to 250
All urls have been extracted successfully


In [29]:
movies = scraper(url)

Extracted 100
Extracted 200


In [30]:
clean_movies = clean_data(movies)

encoded = encode_genre(clean_movies)

df_final = adjust_years(encoded)

df_final

,Year,country,language,production_co,runtime,budget,gross_usa,director_0,director_1,rating,...,Family,Fantasy,History,Music,Musical,Romance,Sci-Fi,Sport,Thriller,War
0,2019,USA,English,Tribeca Productions,209.0,159000000.0,0.0,Martin Scorsese,0,82.0,...,0,0,0,0,0,0,0,0,0,0
1,2019,USA,English,BRON Studios,122.0,55000000.0,332570374.0,Todd Phillips,0,87.0,...,0,0,0,0,0,0,0,0,1,0
2,2019,USA,English,Chernin Entertainment,152.0,97600000.0,94104794.0,James Mangold,0,83.0,...,0,0,0,0,0,0,0,1,0,0
3,2019,UK,English,Heyday Films,136.0,18600000.0,0.0,Noah Baumbach,0,83.0,...,0,0,0,0,0,1,0,0,0,0
4,2019,South Korea,Korean,Barunson E&A,132.0,0.0,19717719.0,Bong Joon Ho,0,86.0,...,0,0,0,0,0,0,0,0,1,0
5,2019,USA,English,Marvel Studios,181.0,356000000.0,858373000.0,Anthony Russo,Joe Russo,85.0,...,0,0,0,0,0,0,1,0,0,0
6,1990,USA,English,Warner Bros.,146.0,25000000.0,46836214.0,Martin Scorsese,0,87.0,...,0,0,0,0,0,0,0,0,0,0
7,1972,USA,English,Paramount Pictures,175.0,6000000.0,134966411.0,Francis Ford Coppola,0,92.0,...,0,0,0,0,0,0,0,0,0,0
8,1947,USA,English,Liberty Films (II),130.0,3180000.0,0.0,Frank Capra,0,86.0,...,1,1,0,0,0,0,0,0,0,0
9,1988,USA,English,Twentieth Century Fox,132.0,28000000.0,83519093.0,John McTiernan,0,82.0,...,0,0,0,0,0,0,0,0,1,0


### Exporting to a CSV

In [31]:
df_final.to_csv("df_top250.csv")